In [ ]:
!pip install cohere tqdm Annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 725.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 11.7 MB/s eta 0:00:00
  Created wheel for Annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552448 sha256=c938377e02dc8d215140770ef90bb493f592b82154e3bb0288eb48d32b3f32f5
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built Annoy
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4


In [ ]:
!pip install cohere

In [ ]:
import cohere
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex

# Paste your API key here. Remember to not share publicly
api_key = '5JJ1dL5eVL96i8pfa9Z3Et0vexvjTQG1E1XbbxmU'

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

In [ ]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""
# Split into a list of sentences
texts = text.split('.')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts])

In [ ]:
# Get the embeddings
texts = np.array([t.strip(' \n') for t in texts]).tolist()
response = co.embed(
  texts=texts,
).embeddings

embeds = np.array(response)
print(embeds.shape)

(15, 4096)


In [ ]:
# Create the search index, pass the size of embedding
search_index = AnnoyIndex(embeds.shape[1], 'angular')

# Add all the vectors to the search index
for index, embed in enumerate(embeds):
    search_index.add_item(index, embed)

search_index.build(10)
search_index.save('test.ann')

True

In [ ]:
texts

['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan',
 'It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
 'Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
 'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007',
 'Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
 'Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm',
 'Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles',
 'Interstellar uses extensive practical a

In [ ]:
import pandas as pd

def search(query):

  # 1. Get the query's embedding
  query_embed = co.embed(texts=[query]).embeddings[0]

  # 2. Retrieve the nearest neighbors
  similar_item_ids = search_index.get_nns_by_vector(query_embed, n=3,
                                                  include_distances=True)

  similar_text = pd.DataFrame([[]])

  # 3. Format the results
  x=0
  for i in similar_item_ids[0]:
    #print(i,x)
    if x==0:
      similar_text = pd.DataFrame([[texts[i],similar_item_ids[1][x]]])
      #print(similar_text)
    else:
      similar_text = pd.concat([similar_text, pd.DataFrame([[texts[i],similar_item_ids[1][x]]])], axis=0)
      #print(similar_text)
    x+=1

  #print(similar_text[0])
  results = pd.DataFrame(data={'texts': similar_text[0],
                              'distance': similar_item_ids[1]})

  #4. Print and return the results
  print(f"Query:'{query}'\nNearest neighbors:")
  return results

In [ ]:
query = "How much did the film make?"
search(query)

Query:'How much did the film make?'
Nearest neighbors:


,texts,distance
0,The film had a worldwide gross over $677 milli...,0.815969
0,"It stars Matthew McConaughey, Anne Hathaway, J...",1.066813
0,"In the United States, it was first released on...",1.086934


In [ ]:
query = "Tell me about the $$$?"
search(query)

Query:'Tell me about the $$$?'
Nearest neighbors:


,texts,distance
0,The film had a worldwide gross over $677 milli...,1.244105
0,"It stars Matthew McConaughey, Anne Hathaway, J...",1.336169
0,Interstellar is a 2014 epic science fiction fi...,1.377908


In [ ]:
query = "Which actors are involved?"
search(query)

Query:'Which actors are involved?'
Nearest neighbors:


,texts,distance
0,"It stars Matthew McConaughey, Anne Hathaway, J...",0.917693
0,Interstellar is a 2014 epic science fiction fi...,1.179189
0,Principal photography began in late 2013 and t...,1.183855


In [ ]:
query = "How was the movie released?"
search(query)

Query:'How was the movie released?'
Nearest neighbors:


,texts,distance
0,"In the United States, it was first released on...",0.871820
0,The film had a worldwide gross over $677 milli...,1.019180
0,Cinematographer Hoyte van Hoytema shot it on 3...,1.073252


## Reranking

In [ ]:
import cohere as co
API_KEY = '5JJ1dL5eVL96i8pfa9Z3Et0vexvjTQG1E1XbbxmU'
co = cohere.Client(API_KEY)
MODEL_NAME = "rerank-english-02" # another option is rerank-multilingual-02

query = "film gross"

In [ ]:
results = co.rerank(query=query, model=MODEL_NAME, documents=texts, top_n=3)

In [ ]:
results = co.rerank(query=query, model=MODEL_NAME, documents=texts, top_n=3) # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Document: {r.document['text']}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

Document Rank: 1, Document Index: 10
Document: The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014
Relevance Score: 0.92


Document Rank: 2, Document Index: 12
Document: It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics
Relevance Score: 0.11


Document Rank: 3, Document Index: 2
Document: Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind
Relevance Score: 0.03


